In [ ]:
#Script methane data management
library(data.table)
library(dplyr)
library(plyr)
library(tidyr)
library(lubridate)
library(ggplot2)


In [ ]:
# Set the directory 
setwd("C:/Users/Ester/Documents/curso metano")

In [ ]:
# Read the sniffer output
bd=read.table("output_methane.txt",sep=",",header=T) #500 rows & 20 cols
bd_id=unique(bd[,1]) #63 individuals
summary(bd)

In [ ]:
# Read test day file
test=read.table("control.csv",sep=";",header=T)#107 rows & 21 cols
test_id=unique(test[,1])#64 #july and sept
#obtain date from test day records
test$test_date1=dmy(test$test_date)

In [ ]:
# Obtain kgm of protein and fat 

test$kgmfat=test$fat*test$milk/100                  
test$kgmprotein=test$protein*test$milk/100 

In [ ]:
############################Join both tables by ID and closest date###################################################################

# Obtain date from output
bd<-tidyr::separate(data = bd, col ="date",into =  c("day", "month","date","time","z","year"), sep = " ", extra = "merge")
bd$sniffer_date=paste(bd$date, bd$month, bd$year, sep = "/")
bd$sniffer_date1<-dmy(bd$sniffer_date)


# Join the two datatables-
bd_full <- lapply(intersect(bd$cow,test$cow),function(id) {
  d1 <- subset(bd,cow==id)
  d2 <- subset(test,cow==id)
  
  d1$indices <- sapply(as.Date(d1$sniffer_date1),function(d) which.min(abs(as.Date(d2$test_date1) - d)))
  d2$indices <- 1:nrow(d2)
  
  merge(d1,d2,by=c('cow','indices'))
}) ###

bd_full <- do.call(rbind,bd_full) #342
bd_full$indices <- NULL
# Check the number of animals that not joined 
bd_antijoin=anti_join(bd,bd_full,by="cow") #158, 3 individuals
bd_full$dif_days=abs(bd_full$sniffer_date1-bd_full$test_date1)
# check the difference of days ###choose less than 40 days
table(bd_full$dif_days)

In [ ]:
#####################################Calving and milking################################################################
# Obtain days in milking
bd_full=tidyr::separate(data=bd_full,col ="calving_date", into=c('calving_date', 'calving_time'),sep=' ')
bd_full$calving_date1=dmy(bd_full$calving_date)
bd_full$daysinmilking=bd_full$sniffer_date1-bd_full$calving_date1
bd_full$daysinmilking=gsub("[a-z]","",bd_full$daysinmilking)#retain only numbers
bd_full$daysinmilking=as.numeric(bd_full$daysinmilking)

# Check difference of days
table(bd_full$daysinmilking)
# Choose The threshold the difference of days , 365
bd_full1=bd_full %>% filter(daysinmilking <= 365) 
# Obtain week of lactation 
bd_full1$week_lactation=floor(bd_full1$daysinmilking / 7)
# Codify days in milking as levels 
bd_full1<-mutate(bd_full1,state_lactation=case_when(
  daysinmilking < 91 ~ "1",
  daysinmilking > 90 & daysinmilking < 151 ~ "2",
  daysinmilking >150 ~ "3"))
table(bd_full1$state_lactation)

# Codify number of calving in three or four levels 
hist(bd_full1$numpar)
table(bd_full1$numpar)
#If there is data of more than 3 calvings
bd_full1<-mutate(bd_full1,num_calving=case_when(
  numpar <= 1 ~ "1",
  numpar > 1 & numpar < 3  ~ "2",
  numpar > 2 ~ "3"))
table(bd_full1$num_calving)


In [ ]:
##############################Ratio and grams per day #########################################################################
#Ratio mean CH4/CO2
bd_full1$ratioCH4CO2=bd_full1$meanCH4/bd_full1$meanCO2
mean(bd_full1$ratioCH4CO2)
# Obtain grams per day 
# Madsen equation
bd_full1$ECM=bd_full1$milk*(0.25+0.122*bd_full1$fat+0.077*bd_full1$protein)
bd_full1$days_inpregnancy=0
bd_full1$gd_madsen<-(0.714*bd_full1$ratioCH4CO2)*180*24*0.001*(5.6*bd_full1$weight**0.75+22*bd_full1$ECM+
                                                                 1.6*0.00001*bd_full1$days_inpregnancy**3)
mean(bd_full1$gd_madsen,na.rm=T)
  
# Tier 2 equation
Cf<-0.386*1.2 #coefficient for feeding situation, lactating
Ca<-0 #Coefficient for activity
Cp<-0.10 #coefficient for pregnancy
DE<-80 #Digestible energy per gross energy in cows fed low quality forage and concentrate

NEm<-Cf*bd_full1$weight**0.75
NEa<-NEm*Ca
NEl<-bd_full1$milk*(1.47+0.4*bd_full1$fat)
NEp<-ifelse(bd_full1$daysinmilking>100, Cp*NEm,0)
REM<-1.123-(0.004092)*DE+0.00001126*(DE)**2-25.4/DE
GE<- 1000*((NEm+NEa+NEl+NEp)/REM)/(DE/100) #g/d
bd_full1$CH4_tier2<-GE*0.065/55.65

mean(bd_full1$CH4_tier2,na.rm=T)

In [ ]:
# Distribution of data
summary(bd_full1)
# Plot the phenotypes distribution 

# By trait
ggplot(bd_full1, aes(x = meanCH4)) +
  geom_histogram( fill = "skyblue", color = "black") +
  theme_minimal() +
  labs(title = "distribution of phenotypes", x = "meanCH4(ppm)", y = "Frequency")

# All traits 
names(bd_full1)
# Convert to long format  (columns to rows)
df_plot <- bd_full1 %>%
  pivot_longer(cols = c(9:23,49,52,53), names_to = "Variable", values_to = "Valor")

ggplot(df_plot, aes(x = Valor)) +
  geom_histogram( fill = "skyblue", color = "black") +
  facet_wrap(~ Variable, scales = "free") +
  theme_minimal() +
  labs(title = "distribution of phenotypes", x = "value", y = "Frequency")

In [ ]:
# Outlier detection by quartiles
ggplot(df_plot, aes(x = Valor)) +
  geom_boxplot(fill = "skyblue", color = "black", outlier.color = "red", outlier.shape = 16) +
  facet_wrap(~ Variable, scales = "free")+
  theme_minimal() +
  labs(title = "Box and whisker plot", x = "Variable", y = "Values")

In [ ]:
##################################################Pattern by hour#####################################################
# Plot the distribution by hour 
bd_full1$hour=substr(bd_full1$time, 1, 2) #susbtract the hour to plot
bd_full1$hour=as.numeric(bd_full1$hour)
table(bd_full1$hour)

# By individual
animal_9765=subset(bd_full1,bd_full1$cow=="9765")
ggplot(data=animal_9765, aes(x= hour, y=gd_madsen)) +
  geom_line()+geom_point()+
  geom_smooth(method = "loess",  span = 0.2,se=TRUE)+
  facet_wrap(facets = vars(date))+
  ggtitle("9765")


# All individuals
ggplot(data=bd, aes(x= hour, y=gd_madsen)) +
  # geom_point() +  # Puntos individuales
  geom_line()+    #línea de tendencia
  geom_smooth(method = "loess",  span = 0.3,se=TRUE)+ 
  facet_wrap(facets = vars(cow))+
  ggtitle("Pattern by hour")

In [ ]:
############################################################DATA FILTERING##########################################################################################
# Threshold by CO2 concentration
bd_full2 <- bd_full1 %>%
  dplyr::mutate(across(c(meanCH4, meanCH4_5s,meanCO2,meanRatioCH4_CO2,AUC_CH4,AUC_Ratio,
                         Sum_of_PeaksCH4,Sum_of_PeaksCH4_5s,Sum_of_PeaksCO2,Sum_of_PeaksRatio,
                         Mean_of_PeaksCH4,Mean_of_PeaksRatio,Sum_MaxPeak, ratioCH4CO2, gd_madsen,
                         CH4_tier2), ~ ifelse(meanCO2 < 2500, NA, .)))

colSums(is.na(bd_full1))
colSums(is.na(bd_full2))


# Make correction by SD
# Function for correction of data ±3 SD
function_outliers <- function(x) {
  mean_x <- mean(x, na.rm = TRUE)
  sd_x <- sd(x, na.rm = TRUE)
  
  # Set upper and lower lim (± 3 SD)
  lower_lim <- mean_x - 3 * sd_x
  upper_lim <- mean_x + 3 * sd_x
  
  # Replace out-of-limit values with Nas 
  x <- ifelse(x < lower_lim | x > upper_lim, NA, x)
  
  return(x)
}

data_corrected <- bd_full2 %>%
  dplyr::mutate(across(
    c(meanCH4, meanCH4_5s,meanCO2,meanRatioCH4_CO2,AUC_CH4,AUC_Ratio,
      Sum_of_PeaksCH4,Sum_of_PeaksCH4_5s,Sum_of_PeaksCO2,Sum_of_PeaksRatio,
      Mean_of_PeaksCH4,Mean_of_PeaksRatio,Sum_MaxPeak, ratioCH4CO2, gd_madsen ,
      CH4_tier2), #Apply the function to these variables  
    function_outliers    # Function to replace outliers
  ))
colSums(is.na(data_corrected))


In [ ]:
#Printing cows numbers
cow_ids<- unique(event_ch4$cow)
print(cow_ids)


In [ ]:
#################### function to read the events of n cow ##################
# Function to get all events of a specific cow
get_unique_cow_event_ids <- function(cow_number) {
  cow_events <- subset(event_ch4, cow == cow_number)
  unique_event_ids <- unique(cow_events$eventID)
  return(unique_event_ids)
}

cow_number <- 72
unique_event_ids <- get_unique_cow_event_ids(cow_number)
print(unique_event_ids)


In [ ]:
############# lets plot an event of a cow   ####### 

# Choose a specific cow and event for the plot
cow_id <- 72
event_id <- 138

plot_data <- subset(event_ch4, cow == cow_id & eventID == event_id)

# Create ggplot plot
p <- ggplot(plot_data, aes(x = datetime, y = ch4)) +
  geom_line() +
  geom_point() +
  labs(title = paste("CH4 Time Series for Cow", cow_id, "in Event", event_id),
       x = "Datetime",
       y = "Ch4") +
  theme_minimal()

# Explicitly print and assign a name to the plot
print(p)


In [ ]:
# Function to detect peaks
detectPeaks <- function(ch4, windowSize = 5, threshold = 0.0005) {
  peaksIni <- c()
  peaksFin <- c()
  stage <- FALSE
  peak_decreasing <- FALSE
  
  for (i in 1:(length(ch4) - windowSize)) {
    tempW <- ch4[i:(i + windowSize - 1)]
    tempX <- 1:windowSize
    
    mean_tempW <- mean(tempW, na.rm = TRUE)
    mean_tempX <- mean(tempX, na.rm = TRUE)
    
    cov_xy <- sum((tempW - mean_tempW) * (tempX - mean_tempX), na.rm = TRUE)
    var_x <- sum((tempX - mean_tempX) * (tempX - mean_tempX), na.rm = TRUE)
    
    pendiente <- cov_xy / var_x
    
    if (pendiente > threshold && !stage) {
      peaksIni <- c(peaksIni, i)
      stage <- TRUE
      peak_decreasing <- FALSE
    } else if (stage && pendiente < -threshold) {
      peak_decreasing <- TRUE
    } else if (stage && peak_decreasing && pendiente > -threshold / 2) {
      peaksFin <- c(peaksFin, i)
      stage <- FALSE
      peak_decreasing <- FALSE
    } else if (length(peaksIni) == 0 && pendiente > threshold / 2) {
      # Handle case where no peaks have been detected yet
    } else if (i == (length(ch4) - windowSize) && peak_decreasing) {
      peaksFin <- c(peaksFin, i)
    }
  }
  
  list(peaksIni = peaksIni, peaksFin = peaksFin)
}

# Apply the peak detection to the entire database
peak_det <- event_ch4 %>%
  group_by(eventID) %>%
  summarise(num_peaks = length(detectPeaks(ch4)$peaksIni))

# Print the results
head(peak_det)
summary(peak_det)


In [ ]:
######################## EXERCISE #######################
#1)Calculate the num of peaks per  minute
    #HINT: Use merge, and lenght of event (visit) 

In [ ]:
# Function to detect peaks and calculate sum2maxpeaks
detectPeaksAndSum <- function(ch4, windowSize = 5, threshold = 0.0005) {
  peaksIni <- c()
  peaksFin <- c()
  stage <- FALSE
  peak_decreasing <- FALSE
  
  for (i in 1:(length(ch4) - windowSize)) {
    tempW <- ch4[i:(i + windowSize - 1)]
    tempX <- 1:windowSize
    
    mean_tempW <- mean(tempW, na.rm = TRUE)
    mean_tempX <- mean(tempX, na.rm = TRUE)
    
    cov_xy <- sum((tempW - mean_tempW) * (tempX - mean_tempX), na.rm = TRUE)
    var_x <- sum((tempX - mean_tempX) * (tempX - mean_tempX), na.rm = TRUE)
    
    pendiente <- cov_xy / var_x
    
    if (pendiente > threshold && !stage) {
      peaksIni <- c(peaksIni, i)
      stage <- TRUE
      peak_decreasing <- FALSE
    } else if (stage && pendiente < -threshold) {
      peak_decreasing <- TRUE
    } else if (stage && peak_decreasing && pendiente > -threshold / 2) {
      peaksFin <- c(peaksFin, i)
      stage <- FALSE
      peak_decreasing <- FALSE
    } else if (length(peaksIni) == 0 && pendiente > threshold / 2) {
      # Handle case where no peaks have been detected yet
    } else if (i == (length(ch4) - windowSize) && peak_decreasing) {
      peaksFin <- c(peaksFin, i)
    }
  }
  
  sum2maxpeaks <- 0
  if (length(peaksIni) > 0 && length(peaksFin) > 0) {
    for (j in 1:length(peaksIni)) {
      start <- peaksIni[j]
      end <- ifelse(j <= length(peaksFin), peaksFin[j], NA)
      if (!is.na(end) && end > start) {
        peak_values <- ch4[start:end]
        max_values <- sort(peak_values, decreasing = TRUE)[1:2]
        avg_max_values <- mean(max_values)
        sum2maxpeaks <- sum2maxpeaks + avg_max_values
      }
    }
  }
  
  list(peaksIni = peaksIni, peaksFin = peaksFin, sum2maxpeaks = sum2maxpeaks)
}

# Apply the peak detection to the entire database
peak_phen <- event_ch4 %>%
  group_by(eventID) %>%
  summarise(
    num_peaks = {
      result <- detectPeaksAndSum(ch4)
      length(result$peaksIni)
    },
    sum2maxpeaks = {
      result <- detectPeaksAndSum(ch4)
      result$sum2maxpeaks
    }
  )

# Print the results
head(peak_phen)

In [ ]:
# Function to detect peaks and calculate sum2maxpeaks and area under the curve
detectPeaksAndSum <- function(ch4, windowSize = 5, threshold = 0.0005) {
  peaksIni <- c()
  peaksFin <- c()
  stage <- FALSE
  peak_decreasing <- FALSE
  
  for (i in 1:(length(ch4) - windowSize)) {
    tempW <- ch4[i:(i + windowSize - 1)]
    tempX <- 1:windowSize
    
    mean_tempW <- mean(tempW, na.rm = TRUE)
    mean_tempX <- mean(tempX, na.rm = TRUE)
    
    cov_xy <- sum((tempW - mean_tempW) * (tempX - mean_tempX), na.rm = TRUE)
    var_x <- sum((tempX - mean_tempX) * (tempX - mean_tempX), na.rm = TRUE)
    
    pendiente <- cov_xy / var_x
    
    if (pendiente > threshold && !stage) {
      peaksIni <- c(peaksIni, i)
      stage <- TRUE
      peak_decreasing <- FALSE
    } else if (stage && pendiente < -threshold) {
      peak_decreasing <- TRUE
    } else if (stage && peak_decreasing && pendiente > -threshold / 2) {
      peaksFin <- c(peaksFin, i)
      stage <- FALSE
      peak_decreasing <- FALSE
    } else if (length(peaksIni) == 0 && pendiente > threshold / 2) {
      # Handle case where no peaks have been detected yet
    } else if (i == (length(ch4) - windowSize) && peak_decreasing) {
      peaksFin <- c(peaksFin, i)
    }
  }
  
  sum2maxpeaks <- 0
  area_under_curve <- 0
  if (length(peaksIni) > 0 && length(peaksFin) > 0) {
    for (j in 1:length(peaksIni)) {
      start <- peaksIni[j]
      end <- ifelse(j <= length(peaksFin), peaksFin[j], NA)
      if (!is.na(end) && end > start) {
        peak_values <- ch4[start:end]
        max_values <- sort(peak_values, decreasing = TRUE)[1:2]
        avg_max_values <- mean(max_values)
        sum2maxpeaks <- sum2maxpeaks + avg_max_values
        
        # Calculate area under the curve above ground level (assumed to be the minimum value in ch4)
        ground_level <- min(ch4)
        area_under_curve <- area_under_curve + sum(peak_values - ground_level)
      }
    }
  }
  
  list(peaksIni = peaksIni, peaksFin = peaksFin, sum2maxpeaks = sum2maxpeaks, area_under_curve = area_under_curve)
}

# Apply the peak detection to the entire database
AUC <- event_ch4 %>%
  group_by(eventID) %>%
  summarise(
    num_peaks = {
      result <- detectPeaksAndSum(ch4)
      length(result$peaksIni)
    },
    sum2maxpeaks = {
      result <- detectPeaksAndSum(ch4)
      result$sum2maxpeaks
    },
    area_under_curve = {
      result <- detectPeaksAndSum(ch4)
      result$area_under_curve
    }
  )

# Print the results
head(AUC)

In [ ]:
############################### now for the 'cut' visit ###########################

# Calculate the time difference in seconds from the first TimeStamp per eventID
event_ch42<- event_ch4 %>%
  group_by(eventID) %>%
  mutate(time_diff_sec = as.numeric(difftime(datetime, min(datetime), units = "secs"))) %>%
  ungroup()

# Function to detect peaks and calculate sum2maxpeaks and area under the curve
detectPeaksAndSum <- function(ch4, windowSize = 5, threshold = 0.0005) {
  peaksIni <- c()
  peaksFin <- c()
  stage <- FALSE
  peak_decreasing <- FALSE
  
  for (i in 1:(length(ch4) - windowSize)) {
    tempW <- ch4[i:(i + windowSize - 1)]
    tempX <- 1:windowSize
    
    mean_tempW <- mean(tempW, na.rm = TRUE)
    mean_tempX <- mean(tempX, na.rm = TRUE)
    
    cov_xy <- sum((tempW - mean_tempW) * (tempX - mean_tempX), na.rm = TRUE)
    var_x <- sum((tempX - mean_tempX) * (tempX - mean_tempX), na.rm = TRUE)
    
    pendiente <- cov_xy / var_x
    
    if (pendiente > threshold && !stage) {
      peaksIni <- c(peaksIni, i)
      stage <- TRUE
      peak_decreasing <- FALSE
    } else if (stage && pendiente < -threshold) {
      peak_decreasing <- TRUE
    } else if (stage && peak_decreasing && pendiente > -threshold / 2) {
      peaksFin <- c(peaksFin, i)
      stage <- FALSE
      peak_decreasing <- FALSE
    } else if (length(peaksIni) == 0 && pendiente > threshold / 2) {
      # Handle case where no peaks have been detected yet
    } else if (i == (length(ch4) - windowSize) && peak_decreasing) {
      peaksFin <- c(peaksFin, i)
    }
  }
  
  sum2maxpeaks <- 0
  area_under_curve <- 0
  if (length(peaksIni) > 0 && length(peaksFin) > 0) {
    for (j in 1:length(peaksIni)) {
      start <- peaksIni[j]
      end <- ifelse(j <= length(peaksFin), peaksFin[j], NA)
      if (!is.na(end) && end > start) {
        peak_values <- ch4[start:end]
        max_values <- sort(peak_values, decreasing = TRUE)[1:2]
        avg_max_values <- mean(max_values)
        sum2maxpeaks <- sum2maxpeaks + avg_max_values
        
        # Calculate area under the curve above ground level (assumed to be the minimum value in ch4)
        ground_level <- min(ch4)
        area_under_curve <- area_under_curve + sum(peak_values - ground_level)
      }
    }
  }
  
  list(peaksIni = peaksIni, peaksFin = peaksFin, sum2maxpeaks = sum2maxpeaks, area_under_curve = area_under_curve)
}

# Apply the peak detection to the entire database, considering only data between 60 sec to 300 sec per eventID
peak_phen2 <- event_ch42 %>%
  filter(time_diff_sec >= 60 & time_diff_sec <= 300) %>%
  group_by(eventID) %>%
  summarise(
    num_peaks = {
      result <- detectPeaksAndSum(ch4)
      length(result$peaksIni)
    },
    sum2maxpeaks = {
      result <- detectPeaksAndSum(ch4)
      result$sum2maxpeaks
    },
    area_under_curve = {
      result <- detectPeaksAndSum(ch4)
      result$area_under_curve
    }
  )

# Print the results
head(peak_phen2)
dim(peak_phen2)

In [ ]:
#Adding moving average, play with the window sizes
detectPeaksAndSum <- function(ch4, windowSize = 5, threshold = 0.0005) {
  peaksIni <- c()
  peaksFin <- c()
  stage <- FALSE
  peak_decreasing <- FALSE
  
  for (i in 1:(length(ch4) - windowSize)) {
    tempW <- ch4[i:(i + windowSize - 1)]
    tempX <- 1:windowSize
    
    mean_tempW <- mean(tempW, na.rm = TRUE)
    mean_tempX <- mean(tempX, na.rm = TRUE)
    
    cov_xy <- sum((tempW - mean_tempW) * (tempX - mean_tempX), na.rm = TRUE)
    var_x <- sum((tempX - mean_tempX) * (tempX - mean_tempX), na.rm = TRUE)
    
    pendiente <- cov_xy / var_x
    
    if (pendiente > threshold && !stage) {
      peaksIni <- c(peaksIni, i)
      stage <- TRUE
      peak_decreasing <- FALSE
    } else if (stage && pendiente < -threshold) {
      peak_decreasing <- TRUE
    } else if (stage && peak_decreasing && pendiente > -threshold / 2) {
      peaksFin <- c(peaksFin, i)
      stage <- FALSE
      peak_decreasing <- FALSE
    } else if (length(peaksIni) == 0 && pendiente > threshold / 2) {
      # Handle case where no peaks have been detected yet
    } else if (i == (length(ch4) - windowSize) && peak_decreasing) {
      peaksFin <- c(peaksFin, i)
    }
  }
  
  sum2maxpeaks <- 0
  area_under_curve <- 0
  if (length(peaksIni) > 0 && length(peaksFin) > 0) {
    for (j in 1:length(peaksIni)) {
      start <- peaksIni[j]
      end <- ifelse(j <= length(peaksFin), peaksFin[j], NA)
      if (!is.na(end) && end > start) {
        peak_values <- ch4[start:end]
        max_values <- sort(peak_values, decreasing = TRUE)[1:2]
        avg_max_values <- mean(max_values)
        sum2maxpeaks <- sum2maxpeaks + avg_max_values
        
        # Calculate area under the curve above ground level (assumed to be the minimum value in ch4)
        ground_level <- min(ch4)
        area_under_curve <- area_under_curve + sum(peak_values - ground_level)
      }
    }
  }
  
  list(peaksIni = peaksIni, peaksFin = peaksFin, sum2maxpeaks = sum2maxpeaks, area_under_curve = area_under_curve)
}

# Function to calculate the average of the moving average of CH4 values
calculate_moving_avg <- function(ch4, window_size) {
  moving_avg <- rollmean(ch4, k = window_size, fill = NA)
  mean(moving_avg, na.rm = TRUE)
}

# Apply the peak detection to the entire database, considering only data between 60 sec to 300 sec per eventID
all_phen <- event_ch42 %>%
  filter(time_diff_sec >= 60 & time_diff_sec <= 300) %>%
  group_by(eventID) %>%
  summarise(
    num_peaks = {
      result <- detectPeaksAndSum(ch4)
      length(result$peaksIni)
    },
    sum2maxpeaks = {
      result <- detectPeaksAndSum(ch4)
      result$sum2maxpeaks
    },
    area_under_curve = {
      result <- detectPeaksAndSum(ch4)
      result$area_under_curve
    },
    avg_moving_avg_ch4 = calculate_moving_avg(ch4, window_size = 10) # Change window_size as needed
  )

# Print the results
head(all_phen)
dim(all_phen)

In [ ]:
#calculate all the phenotypes with cut and not cut and calculate correlations among the phenotypes
#HINT1: use merge or join
merged<- merge(event_5low, event_quant, by="eventID")
head(merged)
merged2<- left_join(event_5low, event_3low, by="eventID")
head(merged2)
#HINT2: use cor and choose the method kendall, spearman or pearson
cor(event_5low$avg_ch4, event_3low$avg_ch4, method="kendall", use="complete.obs")
cor(event_means$mean_ch4, event_means$mean_co2, method="pearson", use="complete.obs")